In [1]:
import os, gc, torch, PIL, pickle,json,io

import pandas as pd
from typing import List
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from utils.plot import plot_losses, plot_train_val_evaluators
from PIL import Image
from models.load import get_model_label, get_model_name, get_current_epoch, get_dataset_label, get_trained_model

from data.constants import DEFAULT_REFLACX_LABEL_COLS

import utils.print as print_f
from utils.init import reproducibility, clean_memory_get_device

# from models.load import TrainedModels
## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    mobilenet_baseline = "val_lesion-detection_ap_0_1655_test_lesion-detection_ap_0_1648_epoch50_03-15-2023 16-43-54_lesion_dsetection_baseline_mobilenet"  # mobilenet baseline
    mobilenet_with_fix = "val_lesion-detection_ap_0_1918_test_lesion-detection_ap_0_1903_epoch16_03-16-2023 11-34-10_lesion_dsetection_with_fixation_mobilenet"
    resnet18_baseline = "val_lesion-detection_ap_0_1973_test_lesion-detection_ap_0_2010_epoch22_03-16-2023 19-44-55_lesion_dsetection_baseline_resnet"
    resnet18_with_fix = "val_lesion-detection_ap_0_1951_test_lesion-detection_ap_0_2195_epoch12_03-17-2023 00-31-54_lesion_dsetection_with_fixation_resnet"
    densenet161_baseline = "val_lesion-detection_ap_0_1990_test_lesion-detection_ap_0_2085_epoch5_03-17-2023 08-53-33_lesion_dsetection_baseline_densenet161"
    densenet161_with_fix = "val_lesion-detection_ap_0_2120_test_lesion-detection_ap_0_2104_epoch12_03-17-2023 18-36-01_lesion_dsetection_with_fixation_densenet161"
    efficientnet_b5_baseline = "val_lesion-detection_ap_0_1898_test_lesion-detection_ap_0_2055_epoch5_03-17-2023 23-30-57_lesion_dsetection_baseline_efficientnet_b5"
    efficientnet_b5_with_fix = "val_lesion-detection_ap_0_2117_test_lesion-detection_ap_0_2190_epoch8_03-18-2023 12-29-20_lesion_dsetection_with_fixation_efficientnet_b5"
    efficientnet_b0_baseline = "val_lesion-detection_ap_0_1934_test_lesion-detection_ap_0_1858_epoch10_03-18-2023 23-50-47_lesion_dsetection_baseline_efficientnet_b0"
    efficientnet_b0_with_fix = "val_lesion-detection_ap_0_2191_test_lesion-detection_ap_0_2162_epoch10_03-18-2023 19-38-11_lesion_dsetection_with_fixation_efficientnet_b0"
    # convnext_base_with_fix = "val_lesion-detection_ap_0_2472_test_lesion-detection_ap_0_2637_epoch15_03-23-2023 20-20-07_lesion_dsetection_with_fixation_convnext_base_silent_report"
    convnext_base_with_fix = "val_lesion-detection_ap_0_2610_test_lesion-detection_ap_0_2548_epoch22_03-22-2023 02-55-37_lesion_dsetection_with_fixation_convnext_base"
    convnext_base_baseline = "val_lesion-detection_ap_0_2426_test_lesion-detection_ap_0_2325_epoch20_03-22-2023 11-53-53_lesion_dsetection_baseline_convnext_base"
    convnext_base_with_fix_silent = "val_lesion-detection_ap_0_2405_test_lesion-detection_ap_0_2543_epoch19_03-24-2023 13-57-29_lesion_dsetection_with_fixation_convnext_base_silent_report"
    convnext_base_with_fix_full = "val_lesion-detection_ap_0_2602_test_lesion-detection_ap_0_2499_epoch22_03-24-2023 04-42-21_lesion_dsetection_with_fixation_convnext_base_full_report"
    vgg16_with_fix = "val_lesion-detection_ap_0_2301_test_lesion-detection_ap_0_2186_epoch22_03-20-2023 19-26-02_lesion_dsetection_with_fixation_vgg16"
    vgg16_baseline = "val_lesion-detection_ap_0_2113_test_lesion-detection_ap_0_2068_epoch12_03-21-2023 00-45-24_lesion_dsetection_baseline_vgg16"
    regnet_y_8gf_with_fix = "val_lesion-detection_ap_0_2267_test_lesion-detection_ap_0_2029_epoch12_03-21-2023 11-28-48_lesion_dsetection_with_fixation_regnet_y_8gf"
    regnet_y_8gf_baseline = "val_lesion-detection_ap_0_1883_test_lesion-detection_ap_0_1658_epoch13_03-21-2023 15-22-32_lesion_dsetection_baseline_regnet_y_8gf"

In [4]:
naming_map = {
    TrainedModels.mobilenet_baseline: "mobilenet_baseline",
    TrainedModels.mobilenet_with_fix: "mobilenet_with_fix",
    TrainedModels.resnet18_baseline: "resnet18_baseline",
    TrainedModels.resnet18_with_fix: "resnet18_with_fix",
    TrainedModels.densenet161_baseline: "densenet161_baseline",
    TrainedModels.densenet161_with_fix: "densenet161_with_fix",
    TrainedModels.efficientnet_b5_baseline: "efficientnet_b5_baseline",
    TrainedModels.efficientnet_b5_with_fix: "efficientnet_b5_with_fix",
    TrainedModels.efficientnet_b0_baseline: "efficientnet_b0_baseline",
    TrainedModels.efficientnet_b0_with_fix: "efficientnet_b0_with_fix",
    TrainedModels.convnext_base_with_fix: "convnext_base_with_fix",
    TrainedModels.convnext_base_baseline: "convnext_base_baseline",
    TrainedModels.convnext_base_with_fix_silent: "convnext_base_with_fix_silent",
    TrainedModels.convnext_base_with_fix_full:"convnext_base_with_fix_full",
    TrainedModels.vgg16_with_fix: "vgg16_with_fix",
    TrainedModels.vgg16_baseline: "vgg16_baseline",
    TrainedModels.regnet_y_8gf_with_fix: "regnet_y_8gf_with_fix",
    TrainedModels.regnet_y_8gf_baseline: "regnet_y_8gf_baseline",
}


In [5]:
score_thrs="0.05"

In [6]:
eval_ss = []

for select_model in TrainedModels:
    model_path = select_model.value
    for l in DEFAULT_REFLACX_LABEL_COLS + ["all"]:
        eval_df = pd.read_csv(
            os.path.join(
                "./eval_results",
                f"{model_path}_{l}_{score_thrs}.csv",
            ),
            index_col=0,
        )
        eval_s = eval_df.iloc[0]
        eval_s["model"] = naming_map[select_model]
        eval_s["lesion"] = l
        eval_s["score_threshold"] = score_thrs
        states = json.loads(eval_s["coco_states"])
        for s in states:
            eval_s[s["p_title_str"]] = s["mean_s"]
        eval_ss.append(eval_s)


In [7]:
eval_df = pd.DataFrame(eval_ss)

In [8]:
eval_df

,num_fps,num_fns,num_tps,coco_states,Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",model,...,Average Precision (AP) @[ IoU=0.50:0.95 | area= small | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= small | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 10 ]
0,938,84,115,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.608543,0.738693,0.812864,0.888883,0.762246,mobilenet_baseline,...,-1,-1,0.099477,0.149749,0.386432,0.437186,0.454271,-1,-1,0.437186
0,448,134,143,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.608303,0.740452,0.864081,0.957607,0.792611,mobilenet_baseline,...,-1,-1,0.172026,0.248375,0.408303,0.414440,0.414440,-1,-1,0.414440
0,1092,139,198,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.507418,0.747774,0.879130,0.937494,0.767954,mobilenet_baseline,...,-1,-1,0.114860,0.168546,0.356973,0.404451,0.408309,-1,-1,0.404451
0,643,182,135,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.322668,0.500631,0.713767,0.906087,0.610788,mobilenet_baseline,...,-1,-1,0.051029,0.150158,0.277287,0.291483,0.294006,-1,-1,0.291483
0,269,306,98,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.462120,0.635768,0.800206,0.889389,0.696871,mobilenet_baseline,...,-1,-1,0.044417,0.087871,0.146040,0.149010,0.149010,-1,-1,0.149010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1357,79,198,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.491233,0.657538,0.840353,0.951384,0.735127,regnet_y_8gf_baseline,...,-1,-1,0.171514,0.289170,0.512635,0.570036,0.573646,-1,-1,0.570036
0,2154,107,230,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.462908,0.652350,0.809918,0.951344,0.719130,regnet_y_8gf_baseline,...,-1,-1,0.093429,0.167953,0.429970,0.484570,0.510089,-1,-1,0.484570
0,1073,132,185,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.308623,0.579090,0.760197,0.877924,0.631458,regnet_y_8gf_baseline,...,-1,-1,0.058112,0.171609,0.367508,0.384227,0.384227,-1,-1,0.384227
0,988,227,177,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.381403,0.555747,0.715246,0.861552,0.628487,regnet_y_8gf_baseline,...,-1,-1,0.052436,0.121040,0.256188,0.275743,0.278713,-1,-1,0.275743


In [9]:
eval_df = eval_df[[
    "model",
    "lesion",
    "num_tps",
    "num_fps",
    "num_fns",
    "Sensitivity@ [avgFP=0.5]",
    "Sensitivity@ [avgFP=1]",
    "Sensitivity@ [avgFP=2]",
    "Sensitivity@ [avgFP=4]",
    "mFROC@[0.5,1,2,4]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]",
    " Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 10 ]",
    " Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 10 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]",
]]


In [10]:
eval_df = eval_df.rename(columns={"num_tps": "#TP @[ IoU=0.50      | area=   all | maxDets= 30 ]", "num_fps": "#FP @[ IoU=0.50      | area=   all | maxDets= 30 ]", "num_fns":"#FN @[ IoU=0.50      | area=   all | maxDets= 30 ]"})

In [11]:
eval_df

,model,lesion,#TP @[ IoU=0.50 | area= all | maxDets= 30 ],#FP @[ IoU=0.50 | area= all | maxDets= 30 ],#FN @[ IoU=0.50 | area= all | maxDets= 30 ],Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.75 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ]
0,mobilenet_baseline,Pulmonary edema,115,938,84,0.608543,0.738693,0.812864,0.888883,0.762246,0.454271,0.100805,0.053652,0.094508,0.099462,0.162356,0.086241,0.149749,0.386432,0.437186
0,mobilenet_baseline,Enlarged cardiac silhouette,143,448,134,0.608303,0.740452,0.864081,0.957607,0.792611,0.414440,0.171808,0.134515,0.171435,0.171808,0.236619,0.181183,0.248375,0.408303,0.414440
0,mobilenet_baseline,Consolidation,198,1092,139,0.507418,0.747774,0.879130,0.937494,0.767954,0.408309,0.114949,0.070709,0.109587,0.114832,0.228495,0.093292,0.168546,0.356973,0.404451
0,mobilenet_baseline,Atelectasis,135,643,182,0.322668,0.500631,0.713767,0.906087,0.610788,0.294006,0.051384,0.034454,0.049859,0.050992,0.109484,0.034449,0.150158,0.277287,0.291483
0,mobilenet_baseline,Pleural abnormality,98,269,306,0.462120,0.635768,0.800206,0.889389,0.696871,0.149010,0.044183,0.034317,0.044903,0.044214,0.099201,0.034263,0.087871,0.146040,0.149010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,regnet_y_8gf_baseline,Enlarged cardiac silhouette,198,1357,79,0.491233,0.657538,0.840353,0.951384,0.735127,0.573646,0.171091,0.131742,0.166215,0.171134,0.234437,0.173545,0.289170,0.512635,0.570036
0,regnet_y_8gf_baseline,Consolidation,230,2154,107,0.462908,0.652350,0.809918,0.951344,0.719130,0.510089,0.095578,0.050339,0.090569,0.093340,0.195110,0.073895,0.167953,0.429970,0.484570
0,regnet_y_8gf_baseline,Atelectasis,185,1073,132,0.308623,0.579090,0.760197,0.877924,0.631458,0.384227,0.057707,0.031901,0.056393,0.057708,0.119215,0.046860,0.171609,0.367508,0.384227
0,regnet_y_8gf_baseline,Pleural abnormality,177,988,227,0.381403,0.555747,0.715246,0.861552,0.628487,0.278713,0.052194,0.041420,0.051205,0.052085,0.117501,0.034475,0.121040,0.256188,0.275743


In [12]:
eval_df.to_csv("evaluation.csv")

In [13]:
eval_df[eval_df['lesion'] == 'all'].sort_values('mFROC@[0.5,1,2,4]',ascending=False)

,model,lesion,#TP @[ IoU=0.50 | area= all | maxDets= 30 ],#FP @[ IoU=0.50 | area= all | maxDets= 30 ],#FN @[ IoU=0.50 | area= all | maxDets= 30 ],Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.75 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ]
0,densenet161_baseline,all,849,4920,685,0.597284,0.796250,0.921627,0.966754,0.820479,0.395612,0.104538,0.060573,0.097924,0.103692,0.210625,0.088013,0.149629,0.332777,0.385189
0,convnext_base_with_fix_silent,all,821,3392,713,0.597851,0.782254,0.917028,0.975557,0.818172,0.427377,0.150139,0.095074,0.144533,0.149965,0.249353,0.140911,0.187155,0.386224,0.425488
0,vgg16_with_fix,all,888,4228,646,0.582176,0.784087,0.913663,0.987333,0.816815,0.430155,0.120113,0.069960,0.115104,0.119823,0.220570,0.111336,0.172412,0.389172,0.427477
0,convnext_base_with_fix,all,945,4758,589,0.592796,0.759081,0.906939,0.974605,0.808355,0.491254,0.150343,0.086347,0.142879,0.149067,0.256095,0.138168,0.186925,0.427443,0.477477
0,efficientnet_b0_with_fix,all,842,3755,692,0.580923,0.763835,0.894797,0.971172,0.802682,0.414264,0.123009,0.076214,0.118237,0.122478,0.214829,0.115364,0.184708,0.374034,0.408440
0,vgg16_baseline,all,918,6122,616,0.569839,0.753455,0.897395,0.969999,0.797672,0.443413,0.122350,0.074767,0.116769,0.121941,0.211818,0.112443,0.177676,0.388790,0.437601
0,convnext_base_baseline,all,905,5145,629,0.573535,0.748165,0.901751,0.965384,0.797209,0.457378,0.138026,0.082400,0.130923,0.137361,0.233165,0.131115,0.175081,0.391597,0.446859
0,efficientnet_b5_with_fix,all,819,3710,715,0.566572,0.751588,0.883266,0.960751,0.790544,0.391246,0.117471,0.072073,0.113261,0.116932,0.220080,0.111972,0.168229,0.354105,0.384768
0,convnext_base_with_fix_full,all,1010,6063,524,0.558193,0.730237,0.898569,0.969965,0.789241,0.525052,0.149078,0.080937,0.138865,0.147849,0.249791,0.143288,0.182403,0.442326,0.510332
0,resnet18_with_fix,all,878,5197,656,0.571132,0.734524,0.881214,0.969328,0.789049,0.447970,0.123580,0.070077,0.117872,0.122695,0.222142,0.110252,0.171902,0.390436,0.438190


In [14]:
# eval_df[eval_df['lesion'] == 'all'].sort_values(' Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 10 ]',ascending=False)